In [2]:
import pandas as pd

# Load CSV into a DataFrame
df = pd.read_csv("/Users/abdul/Downloads/Project/photos.csv")

# See first 5 rows
df

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence,blur_hash
0,XMyPniM9LF0,https://unsplash.com/photos/XMyPniM9LF0,https://images.unsplash.com/uploads/1411949294...,2014-09-29 00:08:38.594364,t,4272,2848,1.50,Woman exploring a forest,michellespencer77,...,NaN,NaN,2375421,6967,woman walking in the middle of forest,NaN,NaN,NaN,NaN,L56bVcRRIWMh.gVunlS4SMbsRRxr
1,rDLBArZUl1c,https://unsplash.com/photos/rDLBArZUl1c,https://images.unsplash.com/photo-141633941111...,2014-11-18 19:36:57.08945,t,3000,4000,0.75,Succulents in a terrarium,ugmonk,...,NaN,NaN,13784815,82141,succulent plants in clear glass terrarium,NaN,NaN,NaN,NaN,LvI$4txu%2s:_4t6WUj]xat7RPoe
2,cNDGZ2sQ3Bo,https://unsplash.com/photos/cNDGZ2sQ3Bo,https://images.unsplash.com/photo-142014251503...,2015-01-01 20:02:02.097036,t,2564,1710,1.50,Rural winter mountainside,johnprice,...,NaN,NaN,1302461,3428,rocky mountain under gray sky at daytime,NaN,NaN,NaN,NaN,LhMj%NxvM{t7_4t7aeoM%2M{ozj[
3,iuZ_D1eoq9k,https://unsplash.com/photos/iuZ_D1eoq9k,https://images.unsplash.com/photo-141487280988...,2014-11-01 20:15:13.410073,t,2912,4368,0.67,Poppy seeds and flowers,krisatomic,...,NaN,NaN,2890238,33704,red common poppy flower selective focus phography,NaN,NaN,NaN,NaN,LSC7DirZAsX7}Br@GEWWmnoLWCnj
4,BeD3vjQ8SI0,https://unsplash.com/photos/BeD3vjQ8SI0,https://images.unsplash.com/photo-141700759404...,2014-11-26 13:13:50.134383,t,4896,3264,1.50,Silhouette near dark trees,jonaseriksson,...,NaN,NaN,8704860,49662,trees during night time,NaN,NaN,NaN,NaN,L25|_:V@0hxtI=W;odae0ht6=^NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,c7OrOMxrurA,https://unsplash.com/photos/c7OrOMxrurA,https://images.unsplash.com/photo-159300793778...,2020-06-24 14:12:32.397564,t,4160,6240,0.67,NaN,andyadcon,...,NaN,NaN,1603469,4757,black metal fence during daytime,NaN,NaN,NaN,NaN,L34d_aJ-I:R*tlxGWUjY1y$i$hsm
24996,15IuQ5a0Qwg,https://unsplash.com/photos/15IuQ5a0Qwg,https://images.unsplash.com/photo-159296761254...,2020-06-24 03:00:42.603563,t,6000,4000,1.50,Pearl earrings and seashells,contentpixie,...,Australia,Melbourne,550016,2544,white and brown seashell on white surface,NaN,NaN,NaN,NaN,LAM%_?_NNIH?xvRPx]kBajRPWAxv
24997,w8nrcXz8pwk,https://unsplash.com/photos/w8nrcXz8pwk,https://images.unsplash.com/photo-159299937329...,2020-06-24 11:53:00.668613,t,2584,4592,0.56,NaN,maur1ts,...,Tansania,NaN,500831,3923,leopard on brown tree trunk during daytime,NaN,NaN,NaN,NaN,LlK1wK00M{%MxvV@x[tRM|oyt8t7
24998,n1jHrRhehUI,https://unsplash.com/photos/n1jHrRhehUI,https://images.unsplash.com/photo-159192792878...,2020-06-12 02:13:04.409162,t,3533,4824,0.73,Floral truck in the streets of Rome,keithalva,...,Italy,Rome,335692,1734,woman in beige coat and white hat standing on ...,NaN,NaN,NaN,NaN,LOIhKfV@0J%N~WM{sT-=g4M{Mxx]


In [32]:
img_url = df.loc[9660, "photo_image_url"]
img_url

'https://images.unsplash.com/photo-1447767876721-7093b7e87667'

In [1]:
from transformers import CLIPProcessor, CLIPModel

# use safetensors version
model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32",
    use_safetensors=True
)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# from transformers import CLIPProcessor, CLIPModel
# from PIL import Image
# import requests

# # 1. Load CLIP model + processor
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

from transformers import CLIPProcessor, CLIPModel

# use safetensors version
model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32",
    use_safetensors=True
)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


# 2. Pick one image from your dataframe (row 0 here)
img_url = img_url
image = Image.open(requests.get(img_url, stream=True).raw)

# 3. Define candidate labels (your tags)
labels = [
    # "a photo of a sea",
    # "a photo of a waves",
    # "a photo of a dog",
    # "a photo of a ocean",
    # "a photo of a sunset",
    # "relaxing",
    "fun",
    # "exciting",
    # "interesting",
    # "surfing",
    # "skiing",
    # "hiking",
    # "museum",
    # "swimming",
    # "clouds",
    "hiking",
    "chilling",
    "meditation"

]

# 4. Preprocess inputs
inputs = processor(text=labels, images=image, return_tensors="pt", padding=True)

# 5. Run the model
outputs = model(**inputs)

# 6. Extract similarity scores
logits_per_image = outputs.logits_per_image   # similarity image <-> text
probs = logits_per_image.softmax(dim=1)       # convert to probabilities

# 7. Print results
for label, prob in zip(labels, probs[0].tolist()):
    print(f"{label}: {prob:.2%}")


fun: 15.51%
hiking: 18.19%
chilling: 5.30%
meditation: 61.00%
